In [85]:
from sktime.datasets import load_from_tsfile
import os
import numpy as np

In [86]:
### get current path
from pathlib import Path
path = Path.cwd()
path

PosixPath('/Users/mojtabaaskarzadeh/Library/CloudStorage/GoogleDrive-ma00048@mix.wvu.edu/My Drive/1-time-series classification in manufacturing/Code')

In [87]:
path_dir = '/Users/mojtabaaskarzadeh/Library/CloudStorage/GoogleDrive-ma00048@mix.wvu.edu/My Drive/1-time-series classification in manufacturing/Datasets'

In [88]:
archive = 'DuckDuckGeese'
print(archive)

DuckDuckGeese


In [89]:
PATH = os.path.join(path_dir,archive)
PATH

'/Users/mojtabaaskarzadeh/Library/CloudStorage/GoogleDrive-ma00048@mix.wvu.edu/My Drive/1-time-series classification in manufacturing/Datasets/DuckDuckGeese'

In [90]:
X_train, y_train = load_from_tsfile(os.path.join(PATH, f"{archive}_TRAIN.ts"), return_data_type="numpy3d")
X_test, y_test = load_from_tsfile(os.path.join(PATH, f"{archive}_TEST.ts"), return_data_type="numpy3d")
X_train.shape, y_train.shape, X_test.shape, y_test.shape

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sktime/datasets/_data_io.py:924: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sktime/datasets/_data_io.py:924: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sktime/datasets/_data_io.py:924: PerformanceWarning:

((50, 1345, 270), (50,), (50, 1345, 270), (50,))

In [91]:
##Swzp axis
#X_train = np.swapaxes(X_train, 1,2)
#X_test = np.swapaxes(X_test, 1,2)
#X_train.shape

In [92]:
ts = np.concatenate((X_train, X_test), axis=0)
print(f"\n The dataset shape is:{ts.shape}")
print(f"\n The number of data samples (N) is:{ts.shape[0]}")
print(f"\n The number of TS length (T) is:{ts.shape[2]}")
print(f"\n The number of TS dimention (M) is:{ts.shape[1]}")


 The dataset shape is:(100, 1345, 270)

 The number of data samples (N) is:100

 The number of TS length (T) is:270

 The number of TS dimention (M) is:1345


In [93]:
import sktime
import os
import numpy as np
import argparse
import time

In [94]:
parser = argparse.ArgumentParser()

# dataset settings
parser.add_argument('--data_path', type=str, default='/Users/mojtabaaskarzadeh/Library/CloudStorage/GoogleDrive-ma00048@mix.wvu.edu/My Drive/1-time-series classification in manufacturing/Datasets',
                    help='the path of data.')
parser.add_argument('--dataset', type=str, default='Etching_Dataset', #Etching
                    help='time series dataset. Options: See the datasets list')

# Training parameter settings
parser.add_argument('--n_kernel', type=int, default=1000,
                    help= "Number of kernels of the transformer " )
                    



args = parser.parse_args("")


In [95]:
## Classification
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline
from sklearn.utils.multiclass import class_distribution

from sktime.classification.base import BaseClassifier
from sktime.transformations.panel.rocket import Rocket

In [96]:
class ROCKETClassifier(BaseClassifier):
    """Classifier wrapped for the ROCKET transformer using RidgeClassifierCV.

    Parameters
    ----------
    num_kernels             : int, number of kernels for ROCKET transform
    (default=10,000)
    n_jobs                  : int, optional (default=1)
    The number of jobs to run in parallel for both `fit` and `predict`.
    ``-1`` means using all processors.
    random_state            : int or None, seed for random, integer,
    optional (default to no seed)

    Attributes
    ----------
    classifier              : ROCKET classifier
    n_classes               : extracted from the data

    Notes
    -----
    @article{dempster_etal_2019,
      author  = {Dempster, Angus and Petitjean, Francois and Webb,
      Geoffrey I},
      title   = {ROCKET: Exceptionally fast and accurate time series
      classification using random convolutional kernels},
      year    = {2019},
      journal = {arXiv:1910.13051}
    }

    Java version
    https://github.com/uea-machine-learning/tsml/blob/master/src/main/java/
    tsml/classifiers/shapelet_based/ROCKETClassifier.java
    """

    _tags = {
        "capability:multivariate": True,
        "capability:unequal_length": False,
        "capability:missing_values": False,
        "capability:train_estimate": False,
        "capability:contractable": False,
    }

    def __init__(
        self,
        num_kernels=10000,
        n_jobs=1,
        random_state=None,
    ):
        self.num_kernels = num_kernels
        self.n_jobs = n_jobs
        self.random_state = random_state

        self.classifier = None

        self.n_classes = 0
        self.classes_ = []
        self.class_dictionary = {}

        super(ROCKETClassifier, self).__init__()

    def _fit(self, X, y):
        """Build a pipeline containing the ROCKET transformer and RidgeClassifierCV.

        Parameters
        ----------
        X : nested pandas DataFrame of shape [n_instances, 1]
            Nested dataframe with univariate time-series in cells.
        y : array-like, shape = [n_instances] The class labels.

        Returns
        -------
        self : object
        """
        self.n_classes = np.unique(y).shape[0]
        self.classes_ = class_distribution(np.asarray(y).reshape(-1, 1))[0][0]
        for index, classVal in enumerate(self.classes_):
            self.class_dictionary[classVal] = index

        self.classifier = rocket_pipeline = make_pipeline(
            Rocket(
                num_kernels=self.num_kernels,
                random_state=self.random_state,
                n_jobs=self.n_jobs,
            ),
            RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True), verbose=True
        )
        rocket_pipeline.fit(X, y)

        return self

    def _predict(self, X):
        """Find predictions for all cases in X.

        Parameters
        ----------
        X : The training input samples. array-like or pandas data frame.
        If a Pandas data frame is passed, a check is performed that it only
        has one column.
        If not, an exception is thrown, since this classifier does not yet have
        multivariate capability.

        Returns
        -------
        output : array of shape = [n_test_instances]
        """
        return self.classifier.predict(X)

    def _predict_proba(self, X):
        """Find probability estimates for each class for all cases in X.

        Parameters
        ----------
        X : The training input samples. array-like or sparse matrix of shape
        = [n_test_instances, series_length]
            If a Pandas data frame is passed (sktime format) a check is
            performed that it only has one column.
            If not, an exception is thrown, since this classifier does not
            yet have
            multivariate capability.

        Returns
        -------
        output : array of shape = [n_test_instances, num_classes] of
        probabilities
        """
        dists = np.zeros((X.shape[0], self.n_classes))
        preds = self.classifier.predict(X)
        for i in range(0, X.shape[0]):
            dists[i, np.where(self.classes_ == preds[i])] = 1

        return dists

In [97]:
t_total = time.time() ##Start timing

classifier = ROCKETClassifier(num_kernels= args.n_kernel ,n_jobs=-1)
classifier.fit(X_train, y_train)

print("\n The classifier is fitted")

[Pipeline] ............ (step 1 of 2) Processing rocket, total=   1.8s
[Pipeline] . (step 2 of 2) Processing ridgeclassifiercv, total=   0.1s

 The classifier is fitted


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alphas to: original_alphas * n_samples. 
  warnings.warn(


In [98]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report,f1_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print(classification_report(y_test, y_pred))

print(" Finished!")
F1 = f1_score(y_test, y_pred, average='macro')
print(f"the f1 score is:{F1}")

print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

[[4 3 0 1 2]
 [1 6 1 2 0]
 [1 2 3 4 0]
 [0 2 2 6 0]
 [2 0 1 2 5]]
                              precision    recall  f1-score   support

black-bellied_whistling_duck       0.50      0.40      0.44        10
              canadian_goose       0.46      0.60      0.52        10
               greylag_goose       0.43      0.30      0.35        10
           pink-footed_goose       0.40      0.60      0.48        10
  white-faced_whistling_duck       0.71      0.50      0.59        10

                    accuracy                           0.48        50
                   macro avg       0.50      0.48      0.48        50
                weighted avg       0.50      0.48      0.48        50

 Finished!
the f1 score is:0.4774720090934924
Total time elapsed: 2.8833s
